<a href="https://colab.research.google.com/github/TomohiroYazaki/MoA/blob/master/MoA_XGBoost_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/fchmiel/xgboost-baseline-multilabel-classification

In [ ]:
!pip show xgboost
!pip uninstall --yes xgboost
!pip install xgboost
!pip show xgboost

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

from xgboost import XGBClassifier
from sklearn.model_selection import KFold
#from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputClassifier

import os
import warnings
#warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [8]:
SEED = 42
NFOLDS = 5
DATA_DIR = '/content/drive/My Drive/Kaggle/MoA/input/'
np.random.seed(SEED)

In [9]:
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')

test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')

# drop id col
X = train.iloc[:,1:].to_numpy()
X_test = test.iloc[:,1:].to_numpy()
y = targets.iloc[:,1:].to_numpy() 

In [10]:
print(X.shape)
print(X_test.shape)

(23814, 875)
(3982, 875)


In [11]:
print(X)

[['trt_cp' 24 'D1' ... 0.2139 0.3801 0.4176]
 ['trt_cp' 72 'D1' ... 0.1241 0.6077 0.7371]
 ['trt_cp' 48 'D1' ... -0.2187 -1.408 0.6931]
 ...
 ['ctl_vehicle' 48 'D2' ... 0.7592 0.6656 0.3808]
 ['trt_cp' 24 'D1' ... 0.7015 -0.629 0.074]
 ['trt_cp' 72 'D1' ... -0.4775 -2.15 -4.252]]


In [12]:
print(X_test)

[['trt_cp' 24 'D1' ... -0.0502 0.151 -0.775]
 ['trt_cp' 72 'D1' ... -0.4764 -1.381 -0.73]
 ['ctl_vehicle' 24 'D1' ... 1.016 0.4924 -0.1942]
 ...
 ['trt_cp' 72 'D1' ... 0.5888 -0.4205 -0.1504]
 ['trt_cp' 48 'D2' ... 0.5074 0.743 -0.0484]
 ['trt_cp' 72 'D1' ... 0.0757 -0.1356 0.528]]


In [13]:

print(X.shape[1])
for i in range(X.shape[1]):
    if type(X[0,i]) is str:
        le=LabelEncoder()
        le.fit(X[:,i])
        X[:,i]=le.transform(X[:,i])
        X_test[:,i]=le.transform(X_test[:,i])

#for x in X[0,:]:
#    print(type(x))


875


In [15]:
print(X)

[[1 24 0 ... 0.2139 0.3801 0.4176]
 [1 72 0 ... 0.1241 0.6077 0.7371]
 [1 48 0 ... -0.2187 -1.408 0.6931]
 ...
 [0 48 1 ... 0.7592 0.6656 0.3808]
 [1 24 0 ... 0.7015 -0.629 0.074]
 [1 72 0 ... -0.4775 -2.15 -4.252]]


In [16]:
print(X_test)

[[1 24 0 ... -0.0502 0.151 -0.775]
 [1 72 0 ... -0.4764 -1.381 -0.73]
 [0 24 0 ... 1.016 0.4924 -0.1942]
 ...
 [1 72 0 ... 0.5888 -0.4205 -0.1504]
 [1 48 1 ... 0.5074 0.743 -0.0484]
 [1 72 0 ... 0.0757 -0.1356 0.528]]


In [17]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

#clf = Pipeline([('encode', CountEncoder(cols=[0, 2])),
#                ('classify', classifier)
#               ])
clf = Pipeline([
                ('classify', classifier)
               ])

In [18]:
params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)

In [ ]:
start = time.time()

In [ ]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((test.shape[0], y.shape[1]))
oof_losses = []
kf = KFold(n_splits=NFOLDS)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    # drop where cp_type==ctl_vehicle (baseline)
    ctl_mask = X_train[:,0]=='ctl_vehicle'
    X_train = X_train[~ctl_mask,:]
    y_train = y_train[~ctl_mask]
    
    clf.fit(X_train, y_train)
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

In [ ]:
learning_time = time.time() - start
print("Learning time:{0}".format(learning_time) + "[sec]")

In [ ]:
# set control train preds to 0
control_mask = train['cp_type']=='ctl_vehicle'
oof_preds[control_mask] = 0

print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(oof_preds)))

In [ ]:
# set control test preds to 0
control_mask = test['cp_type']=='ctl_vehicle'

test_preds[control_mask] = 0

In [ ]:
# create the submission file
sub.iloc[:,1:] = test_preds
sub.to_csv('submission.csv', index=False)